# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [77]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [78]:
import os
from openai import AzureOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

client = AzureOpenAI(
    api_key=azure_openai_api_key,
    api_version="2023-12-01-preview",
    azure_endpoint=azure_openai_api_endpoint
)

In [79]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [87]:
llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0.0
                        )

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [88]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?"

In [89]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?
Human: What is 1+1?
AI:

> Finished chain.


"1+1 equals 2. It's one of the most basic arithmetic operations, but it's fundamental to all sorts of more complex math. Do you have any other questions or topics you're curious about?"

In [90]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?
Human: What is 1+1?
AI: 1+1 equals 2. It's one of the most basic arithmetic operations, but it's fundamental to all sorts of more complex math. Do you have any other questions or topics you're curious about?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew! You mentioned it when we started our conversation. How can I assist you further today?'

In [41]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?
Human: What is 1+1?
AI: 1+1 equals 2. It's one of the most basic arithmetic operations, but it's fundamental to all sorts of more complex math. Do you have any other questions or topics you're curious about?
Human: What is my name?
AI: Your name is Andrew! You mentioned it when we started our conversation. Is there anything else you'd like to talk about or ask?


In [91]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?\nHuman: What is 1+1?\nAI: 1+1 equals 2. It's one of the most basic arithmetic operations, but it's fundamental to all sorts of more complex math. Do you have any other questions or topics you're curious about?\nHuman: What is my name?\nAI: Your name is Andrew! You mentioned it when we started our conversation. How can I assist you further today?"}

In [99]:
memory = ConversationBufferMemory()

In [93]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [94]:
print(memory.buffer)

Human: Hi
AI: What's up


In [95]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [96]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [97]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [100]:
from langchain.memory import ConversationBufferWindowMemory

In [101]:
memory = ConversationBufferWindowMemory(k=1)               

In [102]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [103]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [104]:
# llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [105]:
conversation.predict(input="Hi, my name is Andrew")

"Hello, Andrew! It's great to meet you. I'm an AI here to chat and help with any questions you might have. How's your day going so far?"

In [106]:
conversation.predict(input="What is 1+1?")

"1+1 equals 2. It's one of the most basic arithmetic operations, but it's fundamental to all sorts of more complex math. Do you enjoy math, or is there something else you're more interested in?"

In [107]:
conversation.predict(input="What is my name?")

"I don't actually know your name since you haven't shared it with me yet. But I'd love to get to know you better! What would you like to talk about?"

## ConversationTokenBufferMemory

In [24]:
#!pip install tiktoken

In [108]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
# llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [111]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [112]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [113]:
from langchain.memory import ConversationSummaryBufferMemory

In [114]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [115]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and mentions they are just hanging out. The AI responds casually, and the human asks what is on the schedule for today.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [74]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [75]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and mentions they are just hanging out. The AI responds casually, and the human asks what is on the schedule today.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For a demo, you could showcase a few different aspects of AI that would be impressive and relevant to your customer. Here are some ideas:\n\n1. **Natural Language Processing (NLP) Capabilities**: Demonstrate how an AI can understand and generate human-like text. You could use a language model to answer questions, summarize articles, or even generate creative writing.\n\n2. **Chatbot Interaction**: Show how an AI-powered chatbot can handle customer service inquiries, provide recommendations, or assist with troubleshooting. You could set up a scenario where the chatbot interacts with a user in real-time.\n\n3. **Image Recognition and Analysis**: If you have access to a computer vision model, you could demonstrate how it can identify objects, people, or even emotions in images. This could be particularly engaging if you use real-time video feed.\n\n4. **Predictive Analytics**: Showcase how AI can be used to predict trends or outcomes based on historical data. This could be particularly r

In [76]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI and mentions they are just hanging out. The AI responds casually, and the human asks what is on the schedule today. The AI outlines a busy schedule including a meeting with the product team, time to work on a LangChain project, and a lunch meeting with a customer to discuss the latest in AI. The human asks for a good demo to show, and the AI suggests several options: demonstrating NLP capabilities, chatbot interaction, image recognition and analysis, predictive analytics, the LangChain project, and voice recognition and synthesis. The AI advises tailoring the demo to the customer's specific interests and needs."}